In [1]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

![avatar](https://zh-v2.d2l.ai/_images/vgg.svg)

VGG作者使用了带有3×3卷积核、填充为1（保持高度和宽度）的卷积层，和带有  2×2  池化窗口、步幅为 2（每个块后的分辨率减半）的最大池化层

VGG神经网络连续连接的几个 VGG 块（在 vgg_block 函数中定义）。其中有超参数变量 conv_arch 。该变量指定了每个VGG块里卷积层个数和输出通道数。全连接模块则与AlexNet中的相同

In [2]:
import torch
from torch import nn
from d2l import torch as d2l

#该函数有三个参数，分别对应于卷积层的数量 num_convs、输入通道的数量 in_channels 和输出通道的数量 out_channels.
def vgg_block(num_convs, in_channels, out_channels):
    layers = []
    for _ in range(num_convs):
        layers.append(nn.Conv2d(in_channels, out_channels,kernel_size=3, padding=1))
        layers.append(nn.ReLU())
        #第一个卷积层输入通道改变 ，后面的n个卷积层输入大小不变 
        in_channels = out_channels
    layers.append(nn.MaxPool2d(kernel_size=2,stride=2))   
    return nn.Sequential(*layers)

In [3]:
conv_arch = ((1, 64), (1, 128), (2, 256), (2, 512), (2, 512))

In [4]:
#下面的代码实现了 VGG-11。可以通过在 conv_arch 上执行 for 循环来简单实现
def vgg(conv_arch):
    conv_blks = []
    in_channels = 1
    # 卷积层部分
    for (num_convs, out_channels) in conv_arch:
        conv_blks.append(vgg_block(num_convs, in_channels, out_channels))
        in_channels = out_channels

    return nn.Sequential(
        *conv_blks, nn.Flatten(),
        # 全连接层部分，这里是应为经历上面conv_arch的五个池化层（5次减半），224*224的图片大小为7*7
        nn.Linear(out_channels * 7 * 7, 4096), nn.ReLU(), nn.Dropout(0.5),
        nn.Linear(4096, 4096), nn.ReLU(), nn.Dropout(0.5),
        nn.Linear(4096, 10))

net = vgg(conv_arch)

In [5]:
X = torch.randn(size=(1, 1, 224, 224))
for blk in net:
    X = blk(X)
    print(blk.__class__.__name__,'output shape:\t',X.shape)

Sequential output shape:	 torch.Size([1, 64, 112, 112])
Sequential output shape:	 torch.Size([1, 128, 56, 56])
Sequential output shape:	 torch.Size([1, 256, 28, 28])
Sequential output shape:	 torch.Size([1, 512, 14, 14])
Sequential output shape:	 torch.Size([1, 512, 7, 7])
Flatten output shape:	 torch.Size([1, 25088])
Linear output shape:	 torch.Size([1, 4096])
ReLU output shape:	 torch.Size([1, 4096])
Dropout output shape:	 torch.Size([1, 4096])
Linear output shape:	 torch.Size([1, 4096])
ReLU output shape:	 torch.Size([1, 4096])
Dropout output shape:	 torch.Size([1, 4096])
Linear output shape:	 torch.Size([1, 10])


In [6]:
#由于VGG-11比AlexNet计算量更大，因此我们构建了一个通道数较少的网络（都除以ratio），足够用于训练Fashion-MNIST数据集
ratio = 4
small_conv_arch = [(pair[0], pair[1] // ratio) for pair in conv_arch]
net = vgg(small_conv_arch)

In [ ]:
# 训练
lr, num_epochs, batch_size = 0.05, 10, 128
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=224)
d2l.train_ch6(net, train_iter, test_iter, num_epochs, lr, d2l.try_gpu())

Mac由于没有GPU，这里通过markdown直接展示结果。

![avatar](https://zh-v2.d2l.ai/_images/output_vgg_4a7574_59_1.svg)

loss 0.173, &ensp; train acc 0.936,  &ensp; test acc 0.924 

2553.9 examples/sec on cuda:0